# Creating the schema of the database in Postgresql

![title](https://ubunlog.com/wp-content/uploads/2018/07/postgresql.jpeg)

In this notebook we will try to build the schema of the postgres database, making the transactions needed for the table creation.

## Connecting to postgres database

Since we are using python, we need to use a library so we can connect to the postgres server and do transactions. IN our case, we are using the `psycopg2` library.

In [1]:
import psycopg2 as psql

We define the parameters of the connection:
- host
- database
- user
- password

In [2]:
conn = psql.connect(
    host="localhost",
    database="test",
    user="postgres",
    password="123"
)

We can ensure the connection is correct by printing the postgesSQL version in the server.

In [3]:
cur = conn.cursor()

print('PostgreSQL database version:')

cur.execute('SELECT version()')
db_version = cur.fetchone()
print(db_version)

cur.close()

PostgreSQL database version:
('PostgreSQL 13.2 (Debian 13.2-1.pgdg100+1) on x86_64-pc-linux-gnu, compiled by gcc (Debian 8.3.0-6) 8.3.0, 64-bit',)


Once we have finished using the database connection, we can close it.

In [4]:
conn.close()

## Creating tables

Now we have our connection established, we want to create all the tables we need. In the  following image, we can see an schema of our tables.

![title](../Images/DB_Design.png)

### Defining commands

First, we are defining each command in a variable. As you can see, we first drop the table if it exists. It allows us to clear the database in case the tables are already created.

In [5]:
percentile_groups_table = '''
    DROP TABLE IF EXISTS percentil_groups CASCADE;
    
    CREATE TABLE percentil_groups (
        id SERIAL PRIMARY KEY,
        project_id VARCHAR(255) NOT NULL,
        metadata JSON,
        number_genes INTEGER NOT NULL,
        number_cells INTEGER NOT NULL
    );
'''

In [6]:
percentiles_table = '''
    DROP TABLE IF EXISTS percentiles CASCADE;
    
    CREATE TABLE percentiles (
        id SERIAL PRIMARY KEY,
        gene_name VARCHAR(255) NOT NULL,
        percentile float NOT NULL,
        percentil_group INTEGER NOT NULL,
        FOREIGN KEY (percentil_group) 
            REFERENCES percentil_groups (id) 
            ON UPDATE CASCADE ON DELETE CASCADE
    );
'''

In [7]:
gcn_table = '''
    DROP TABLE IF EXISTS gcn CASCADE;

    CREATE TABLE gcn (
        id SERIAL PRIMARY KEY,
        project_id VARCHAR(255) NOT NULL,
        correction VARCHAR(255) NOT NULL,
        iter_pseudocells INTEGER NOT NULL,
        metadata JSON
    );
'''

In [8]:
modules_table = '''
    DROP TABLE IF EXISTS modules CASCADE;

    CREATE TABLE modules (
        id SERIAL PRIMARY KEY,
        name VARCHAR(255) NOT NULL,
        gcn INTEGER NOT NULL,
        FOREIGN KEY (gcn) 
            REFERENCES gcn (id) 
            ON UPDATE CASCADE ON DELETE CASCADE
    );
'''

In [9]:
module_membership_table = '''
    DROP TABLE IF EXISTS module_membership CASCADE;

    CREATE TABLE module_membership (
        id SERIAL PRIMARY KEY,
        module INTEGER NOT NULL,
        gene_name VARCHAR(255) NOT NULL,
        MM float NOT NULL,
        FOREIGN KEY (module) 
            REFERENCES modules (id) 
            ON UPDATE CASCADE ON DELETE CASCADE
    );
'''

In [10]:
term_table = '''
    DROP TABLE IF EXISTS term CASCADE;

    CREATE TABLE term (
        id SERIAL PRIMARY KEY,
        name VARCHAR(255) NOT NULL,
        term_id VARCHAR(255) NOT NULL,
        source VARCHAR(255) NOT NULL,
        IC float NULL
    );
'''

In [11]:
annotation_table = '''
    DROP TABLE IF EXISTS annotation CASCADE;

    CREATE TABLE annotation (
        id SERIAL PRIMARY KEY,
        module INTEGER NOT nULL,
        term INTEGER NOT NULL,
        p_value float NOT NULL,
        FOREIGN KEY (module) 
            REFERENCES modules (id) 
            ON UPDATE CASCADE ON DELETE CASCADE,
        FOREIGN KEY (term) 
            REFERENCES term (id) 
            ON UPDATE CASCADE ON DELETE CASCADE
    );
'''

In [12]:
commands = [
    percentile_groups_table,
    percentiles_table,
    gcn_table,
    modules_table,
    module_membership_table,
    term_table,
    annotation_table
]

### Run the commands and create the tables

Lastly, we are executing each command in the postgres database.

In [13]:
try:
    # connect to the PostgreSQL server
    conn = psql.connect(
        host="localhost",
        database="test",
        user="postgres",
        password="123"
    )
    cur = conn.cursor()
    
    # create table one by one
    for command in commands:
        cur.execute(command)
    # close communication with the PostgreSQL database server
    cur.close()
    # commit the changes
    conn.commit()
except (Exception, psql.DatabaseError) as error:
    print(error)
finally:
    if conn is not None:
        conn.close()

In [14]:
try:
    # connect to the PostgreSQL server
    conn = psql.connect(
        host="localhost",
        database="test",
        user="postgres",
        password="123"
    )
    cur = conn.cursor()
    
    # read tables
    cur.execute("""
        SELECT table_name 
        FROM information_schema.tables
        WHERE table_schema = 'public'
    """)
    for table in cur.fetchall():
        print(table)
    # close communication with the PostgreSQL database server
    cur.close()
    # commit the changes
    conn.commit()
except (Exception, psql.DatabaseError) as error:
    print(error)
finally:
    if conn is not None:
        conn.close()

('percentil_groups',)
('percentiles',)
('gcn',)
('modules',)
('module_membership',)
('annotation',)
('term',)


All the tables have been correctly created.